# 本文件说明
- 需求时间：20180731
- 挑出八分类找那个各类数据（title + content）分词计算词频

# 基本设置

In [66]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import jieba
from collections import Counter
import pre_cor_1

# 导入数据

In [67]:
data = pd.read_excel('data/title_content_label.xlsx')
data['label'] = data['label'].apply(lambda x:x.strip())
print(data.shape)
data.head()

(10530, 5)


,title,content,label,title_content,index
0,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,噪音,做好规划不当月光族不做守财奴。 一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···...,0
1,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,噪音,做车险代理的前景如何 谊信合让你的财富事业迈上新高度。 小本创业网\n\r»\n\r加盟项目...,1
2,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,产品销售,坐飞机必须买保险？航意险适合哪些人？。 坐飞机必须买保险？航意险适合哪些人？\n\r 随着...,2
3,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,噪音,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示。 \n\r为推进我县\n“五...,3
4,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,消费服务,昨天车险报案比平常翻倍。 (图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州...,4


In [68]:
data['label'].value_counts()

噪音        3376
消费服务      1573
监管        1488
其他相关报道    1456
公司内部管理     901
行业         763
资本市场       583
产品销售       390
Name: label, dtype: int64

# 导入字典

## 负面词

In [69]:
neg_1 = []
f = open('corpus/neg_words.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    neg_1.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(neg_1))

1147


In [70]:
neg_2 = []
f = open('corpus/negative_words.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    neg_2.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close() 
print(len(neg_2))

864


In [71]:
# neg_3 = pd.read_excel('corpus/银行业微博负面词.xlsx')
# print(neg_3.shape)
# neg_3.head()

In [72]:
# neg_4 = pd.read_excel('corpus/银行业负面词及预警词-长文本.xlsx')
# print(neg_4.shape)
# neg_4.head()

In [73]:
neg_words = neg_1 + neg_2 # + neg_3['关键词'].tolist() + neg_4['预警词'].tolist()
print('去重前：', len(neg_words))
neg_words = list(set(neg_words))
print('去重后：', len(neg_words))\

remove_list = ['银行',]
for remove_word in remove_list:
    if remove_word in  neg_words:
        neg_words.remove(remove_word)
        print('remove_word: ', remove_word)

去重前： 2011
去重后： 1959
remove_word:  银行


In [74]:
# x = '银行'
# if x in neg_words:
#     print(neg_words.index(x))
#     print(neg_words[neg_words.index(x)])

In [75]:
f = open('corpus/neg_words_20180731_insurance.txt', 'w+', encoding = 'utf-8')
for line in neg_words:
    f.write(line + '\n')
f.close()    

## 保险业词典

In [76]:
bank_dict_1 = []
f = open('corpus/business.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    bank_dict_1.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(bank_dict_1))
bank_dict_1[:5]

282


['“奇葩”险', '保单', '保单贷款', '保单过期', '保单门']

In [77]:
bank_dict_2 = []
f = open('corpus/companies_name.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    bank_dict_2.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(bank_dict_2))
bank_dict_2[:5]

1735


['紫金财产保险股份有限公司', '紫金保险', '紫金财产保险', '紫金财险', '珠峰财产保险股份有限公司']

In [78]:
bank_dict_3 = []
f = open('corpus/company.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    bank_dict_3.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    
print(len(bank_dict_3))
bank_dict_3[:5]

1109


['RGA美国再保险公司上海分公司',
 '爱和谊日生同和财产保险（中国）有限公司',
 '爱心人寿保险股份有限公司',
 '安邦保险集团股份有限公司',
 '安邦财产保险股份有限公司']

In [79]:
insurance_company = bank_dict_2 + bank_dict_3
print('去重前：', len(insurance_company))
insurance_company = list(set(insurance_company))
insurance_company = [w for w in insurance_company if len(w) > 1]
print('去重后：', len(insurance_company))

f = open('corpus/insurance_company_20180803.txt', 'w+', encoding = 'utf-8')
for line in insurance_company:
    if len(line) > 0:
        f.write(line + '\n')
f.close()   

去重前： 2844
去重后： 1975


In [80]:
bank_dict_4 = []
f = open('corpus/user_dict.txt', 'r+', encoding = 'utf-8')
for line in f.readlines():
    bank_dict_4.append(line.strip().encode('utf-8').decode('utf-8-sig'))
f.close()    

bank_dict_4 = [b.split(' ')[0] for b in bank_dict_4 if len(b) > 0]
print(len(bank_dict_4))
bank_dict_4[:5]

5504


['监管函', '存问题', '偿付能力下降', '违规股权', '关联交易']

In [81]:
print('去重前：', len(bank_dict_4))
bank_dict_4 = list(set(bank_dict_4))
print('去重后：', len(bank_dict_4))

去重前： 5504
去重后： 3127


In [82]:
import dict_dbutils
bank_dict_5 = list(set(dict_dbutils.get_dicts().keys()))
print(len(bank_dict_5))
bank_dict_5[:5]

1436


['英大人寿', '北京保监局', '东海航运保险股份有限公司', '珠峰财险', '众安在线产险']

In [83]:
bank_dict_6 = pd.read_excel('corpus/180801保监会采集词.xlsx')
print(bank_dict_6.shape)
bank_dict_6.head()

(1154, 4)


,采集词,批次,状态,备注
0,保险,第一批,其他,NaN
1,保监,第一批,监管-机构,NaN
2,险种,第一批,其他,NaN
3,寿险,第一批,其他,NaN
4,保费,第一批,其他,NaN


In [84]:
bank_dict_6_1 = []
for w in bank_dict_6['采集词']:
    if bool(re.search('[a-z]', w)):
        w_l = [w]
        print(w)
    else :
        w_l = w.split()
    bank_dict_6_1 += w_l

print('去重前：', len(bank_dict_6_1))
bank_dict_6_1 = list(set(bank_dict_6_1))
print('去重后：', len(bank_dict_6_1))

Cao Yu
Chen Wenhui 
Guo Shuqing
Huang Hong
Li Xinran
Liang Tao
Wang Zhaoxing
Zhou Liang
Zhu Shumin
insurance
Chinese insurance regulator
去重前： 1177
去重后： 1148


In [91]:
bank_dict_7 = []
filename = 'corpus/insurance_regulators_20180803.txt'
fid = open(filename, "r+", encoding='UTF-8-sig')
for f in fid:
    bank_dict_7.append(f.strip().replace('\n', ''))
fid.close()

print('去重前：', len(bank_dict_7))
bank_dict_7 = list(set(bank_dict_7))
print('去重后：', len(bank_dict_7))

filename = 'corpus/insurance_regulators_20180804.txt'
fid = open(filename, "w+", encoding='UTF-8')
for line in bank_dict_7:
    fid.write(line + '\n')
fid.close()

去重前： 351
去重后： 298


In [86]:
# print(bank_dict_7)

In [87]:
bank_dict = bank_dict_1 + bank_dict_2 + bank_dict_3 + bank_dict_4 + bank_dict_5 + bank_dict_6_1 + bank_dict_7
print('去重前：', len(bank_dict))
bank_dict = list(set(bank_dict))
print('去重后：', len(bank_dict))

# remove_list = ['银行',]
# for remove_word in remove_list:
#     if remove_word in  neg_words:
#         neg_words.remove(remove_word)
#         print('remove_word: ', remove_word)

去重前： 9135
去重后： 3849


In [88]:
f = open('corpus/insurance_dict_20180803.txt', 'w+', encoding = 'utf-8')
for line in bank_dict:
    if len(line) > 0:
        f.write(line + '\n')
f.close()   

## 预处理 + 分词 + 词频统计

In [90]:
with pd.ExcelWriter('data_count/class_word_counts_20180803.xlsx') as writer:
    for label in data['label'].unique():
        print('-- 处理机构：', label)    
        bank_data = data[data['label'] == label]['title_content'].tolist()
        print('---- 共 %s 条数据'%len(bank_data)) 

        bank_pre = pre_cor_1.handle_contents(bank_data)

        word_count = {}
        for words in bank_pre:
            word_list = words.split(' ')
            for word in word_list:
                if len(word) <= 1: #排除单个字符的分词结果
                    continue
                elif str(word) == 'nan':
                    continue
                else :
                    word_count[word]=word_count.get(word,0)+1

        word_data = pd.DataFrame.from_dict(word_count, orient='index').sort_values(by = 0, ascending = False).reset_index()
        word_data.columns = ['word', 'freq']
        word_data.fillna(0).to_excel(writer, label, index = False)
    writer.save() 

-- 处理机构： 噪音
---- 共 3376 条数据
-- 处理机构： 产品销售
---- 共 390 条数据
-- 处理机构： 消费服务
---- 共 1573 条数据
-- 处理机构： 资本市场
---- 共 583 条数据
-- 处理机构： 监管
---- 共 1488 条数据
-- 处理机构： 公司内部管理
---- 共 901 条数据
-- 处理机构： 其他相关报道
---- 共 1456 条数据
-- 处理机构： 行业
---- 共 763 条数据
